In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, sys, time
import argparse
import tarfile
import re
import scipy.stats as stats

##impor  BMD files from directory
##TODO: combine bMD processing to single file, combine LPR processing to single file
import BMD_Analysis_Functions as baf

# Morpho Step 1: Format File

In [2]:
# This is the format_morpho_input function

##############################################
## READ FILE AND SUBSET TO RELEVANT COLUMNS ##
##############################################

# Read morphology file 
df_morph = pd.read_csv('./test_files/7_PAH_zf_morphology_data_2020NOV11_tall_3756.csv', header = 0)

# List relevant column names
relevant_columns = ['chemical.id', 'conc', 'plate.id', 'well', 'endpoint', 'value']

# The input file must absolutely have these columns, no exceptions 
if all(col in df_morph.columns for col in relevant_columns) == False:
    sys.exit(print("The input file", mfiles, "must have the columns:", ', '.join(relevant_columns)))

# Keep only relevant columns
df_morph = df_morph.loc[:,relevant_columns]

##################################
## SUBSET TO RELEVANT ENDPOINTS ##
##################################

# List the relevant endpoints, which is different for BRAIN samples  
if "BRAI" in list(df_morph["endpoint"].unique()):
    relevant_endpoints = ['AXIS', 'BRAI', 'CFIN', 'CIRC', 'DNC_', 'DP24', 'EYE_', 'JAW_', 'MO24', 
                          'MORT', 'NC24', 'NC__', 'OTIC', 'PE__', 'PFIN', 'PIG_', 'SM24', 'SNOU', 
                          'SOMI', 'SWIM', 'TRUN', 'TR__', 'YSE_']
else:
    relevant_endpoints = ['AXIS', 'BRN_', 'CRAN', 'DNC_', 'DP24', 'EDEM', 'LTRK', 'MO24', 'MORT', 
                          'MUSC', 'NC__', 'SKIN','SM24', 'TCHR']
    
# Subset down to the relevant endpoints 
df_morph = df_morph[df_morph["endpoint"].isin(relevant_endpoints)]

###########################
## ADD MISSING ENDPOINTS ##
###########################

def new_endpoint(endpoints, new_name):
    """
    Generate a new endpoint which is a binary "or" of other endpoints,
    meaning that if there is a 1 in any of the other endpoints, the 
    resulting endpoint is a 1. Otherwise, it is 0 unless the other 
    endpoints are all NA. Then the final value is NA.
    
    Attributes
    ----
    endpoints: A list of column names, as strings, to binary "or". 
    new_name: The name of the new endpoint. 
    
    """
    sub_df = df_morph[df_morph["endpoint"].isin(endpoints)]
    sub_df["endpoint"] = new_name
    sub_df = sub_df.groupby(by = ["chemical.id", "conc", "plate.id", "well", "endpoint"], as_index = False).sum()
    sub_df['value'].values[sub_df['value'] > 1] = 1 
    return(sub_df)
    
# New endpoints to add is a smaller list if the sample is not from BRAIN
if "BRAI" in list(df_morph["endpoint"].unique()):
    
    df_morph = pd.concat(
        [new_endpoint(['MO24','DP24','SM24','NC24'], 'ANY24'),
         new_endpoint(['MORT', 'YSE_', 'AXIS', 'EYE_', 'SNOU', 'JAW_', 'OTIC', 'PE__', 'BRAI', 
                      'SOMI', 'PFIN', 'CFIN', 'PIG_', 'CIRC', 'TRUN', 'SWIM', 'NC__', 'TR__', 
                      'ANY24'], 'ANY120'),
         new_endpoint(['MO24','MORT'], 'TOT_MORT'),
         new_endpoint(['DP24','SM24','NC24', 'YSE_', 'AXIS', 'EYE_', 'SNOU', 'JAW_', 'OTIC', 'PE__', 
                      'BRAI', 'SOMI', 'PFIN', 'CFIN', 'PIG_', 'CIRC','TRUN', 'SWIM', 'NC__', 'TR__'], 'ALL_BUT_MORT'),
         new_endpoint(['BRAI','OTIC','PFIN'], 'BRN_'),
         new_endpoint(['EYE_', 'SNOU', 'JAW_'], 'CRAN'),
         new_endpoint(['YSE_','PE__'], 'EDEM'),
         new_endpoint(['TRUN','CFIN'], 'LTRK'),
         new_endpoint(['CIRC','SWIM','SOMI'], 'MUSC'),
         new_endpoint(['PIG_'], 'SKIN'),
         new_endpoint(['TR__'], 'TCHR'),
         df_morph]
    )
    
else:
    
    df_morph = pd.concat(

        # 1. Add any effect at 24hrs (combination of MO24, DP24 and SM24) 
        [new_endpoint(['MO24','DP24','SM24'], 'ANY24'),

        # 2. Any effect within 5 days (combination of all measurements at both time points)
        new_endpoint(['AXIS', 'BRN_', 'CRAN', 'EDEM', 'LTRK', 'MORT', 'MUSC', 'NC__', 'SKIN', 'TCHR', 'ANY24'], 'ANY120'),

        # 3. Total mortality (MO24 + MORT) 
        new_endpoint(['MO24','MORT'], 'TOT_MORT'),

        # 4. Any effect except mortality (#2 minus MO24 and MORT)
        new_endpoint(['AXIS', 'BRN_', 'CRAN', 'DP24', 'EDEM', 'LTRK', 'MUSC', 'NC__', 'SKIN', 'SM24', 'TCHR'], 'ALL_BUT_MORT'),
        
        # Add original dataframe
        df_morph]
    )
    
###########################################
## CALCULATE VARIABLES FOR DOSE RESPONSE ##
###########################################

# Create groups of each chemical id, concentration, and plate id
plate_groups = df_morph.drop(["well"], 1).groupby(by = ["chemical.id", "conc", "plate.id", "endpoint"], as_index = False)

# Get the number of samples per group
num_tot_samples = plate_groups.size().rename(columns = {"size": "num.tot"})

# Get the number of non-na samples per groups
num_nonna = plate_groups.count().rename(columns = {"value": "num.nonna"})

# Get the number affected
num_affected = plate_groups.sum().rename(columns = {"value": "num.affected"})

# Merge to create missingness dataframe
plate_groups = pd.merge(pd.merge(num_tot_samples, num_nonna), num_affected)

# Create IDs of chemical.id, plate.id, and endpoint in plate_groups 
ids = []
for row in range(len(plate_groups)):
    ids.append(str(plate_groups["chemical.id"][row]) + " " + str(plate_groups["plate.id"][row]) + " " + str(plate_groups["endpoint"][row]))
plate_groups["ids"] = ids

#####################################################################
## REMOVE VARIABLES WITH HIGH MISSINGNESS IN BASELINE MEASUREMENTS ##
#####################################################################

# Identify 0 (baseline) concentrations with high missingness (greater than 50% missing or less than 50% non-missing)
missingness = plate_groups.loc[plate_groups["conc"] == 0]
missingness["keep"] = missingness["num.nonna"] / missingness["num.tot"] > 0.5 # TODO: Add a report of what was removed --> txt file "nothing removed"

# Identify plates to keep 
tokeep = missingness.loc[missingness["keep"]]["ids"].tolist()
plate_groups = plate_groups[plate_groups["ids"].isin(tokeep)]

# Stop if everything gets removed
if len(plate_groups) == 0:
    sys.exit("Everything was removed with the 50% missingness filter")

#######################################
## REGROUP WITHOUT PLATE IDS AND SUM ##
#######################################

# First, remove plate.id and ids column
chemical_groups = plate_groups.drop(columns = ["plate.id", "ids"])

# Group by chemical.id, concentration, and endpoint. Then, sum the results. 
chemical_groups = chemical_groups.groupby(by = ["chemical.id", "conc", "endpoint"]).sum().reset_index()

############################
## RETURN FORMATTED TABLE ##
############################
chemical_groups


,chemical.id,conc,endpoint,num.tot,num.nonna,num.affected
0,3756,0.0,ALL_BUT_MORT,36,36,2.0
1,3756,0.0,ANY120,36,36,2.0
2,3756,0.0,ANY24,36,36,4.0
3,3756,0.0,AXIS,36,32,2.0
4,3756,0.0,BRN_,36,32,0.0
...,...,...,...,...,...,...
139,3756,100.0,NC__,36,30,0.0
140,3756,100.0,SKIN,36,30,0.0
141,3756,100.0,SM24,36,32,0.0
142,3756,100.0,TCHR,36,30,0.0


# Morpho Step 2: Generate Dose Response and Write Flags

In [3]:
# This is the generate_dose_response function

dose_response = chemical_groups

'''This function performs feasibility analysis
for dose response data. The value returned is a
flag indicating data quality as defined below:

0: Not enough dose groups for BMD analysis. BMD analysis not performed. BAD.
1: No trend detected in dose-response data. BMD analysis not performed. BAD. 
2: Good dose-response data. BMD analysis is performed. GOOD.
3: Dose-response data quality poor. BMD analysis might be unreliable. GOOD. 
4: Data resolution poor. BMD analysis might be unreliable. MODERATE.
5: Negative correlation. 

'''

########################
## CALCULATE RESPONSE ##
########################

# Add an id column
ids = []
for row in range(len(dose_response)):
    ids.append(str(dose_response["chemical.id"][row]) + " " + str(dose_response["endpoint"][row]))
dose_response["ids"] = ids

# Response is the number affected over the number of embryos (non-na)
dose_response["frac.affected"] = dose_response["num.affected"] / dose_response["num.nonna"]

####################################
## GENERATE QUALITY CONTROL FLAGS ##
####################################

# Count the number of unique concentrations per chemical id and endpoint pairing 
BMD_Flags = dose_response.groupby(["chemical.id", "endpoint", "ids"])["conc"].nunique().reset_index().rename(columns = {"conc": "num.conc"})

# Add a flag category
BMD_Flags["flag"] = None

# If there are less than 3 points, the BMD flag is 0 - not enough dose groups
BMD_Flags["flag"].values[BMD_Flags["num.conc"] < 3] = 0

# Change dose response 
dose_response["conc"] = dose_response["conc"].astype('float') + 1e-15

# Calculate the spearman correlation
spear = dose_response[["chemical.id", "endpoint", "conc", "frac.affected"]].groupby(["chemical.id", "endpoint"]).corr(method = "spearman").unstack().iloc[:,1].reset_index()
spear = spear.set_axis(["chemical.id", "endpoint", "spearman"], axis = 1)

# Merge spearman to the BMD_Flags dataframe
BMD_Flags = BMD_Flags.merge(spear)

# If spearman correlation is below 0.2 or is NaN, the Flag is 1 - no strong trend 
BMD_Flags["flag"].values[(BMD_Flags["spearman"] < 0.2) | (BMD_Flags["spearman"].isna())] = 1

# If the correlation is above 0.25 or below 0.8, run a t-test
BMD_Flags["run.ttest"] = (BMD_Flags["spearman"] >= 0.20) & (BMD_Flags["spearman"] <= 0.80)

# If the correlation is above 0.8, assign the flag at 2 - good
BMD_Flags["flag"].values[BMD_Flags["spearman"] > 0.8] = 2

# Run the t-test only where indicated 
ttest = dose_response[dose_response["ids"].isin(BMD_Flags[BMD_Flags["run.ttest"]]["ids"].to_list())][["ids", "frac.affected"]]
ttest = ttest.groupby("ids").apply(lambda df: stats.ttest_1samp(np.diff(df["frac.affected"]), 0)[1]).reset_index().rename(columns = {0:"ttest.pval"})

# Merge ttest results 
BMD_Flags = BMD_Flags.merge(ttest, on = "ids", how = "outer")

# A p-value of less than 0.05 gets a flag of 2 - good, from 0.05 to 0.32 gets a 3 - unreliable, 
# and greater than 0.32 gets very unreliable. 
BMD_Flags["flag"].values[BMD_Flags["ttest.pval"] <= 0.05] = 2
BMD_Flags["flag"].values[(BMD_Flags["ttest.pval"] > 0.05) & (BMD_Flags["ttest.pval"] <= 0.32)] = 3
BMD_Flags["flag"].values[BMD_Flags["ttest.pval"] > 0.32] = 4

BMD_Flags


,chemical.id,endpoint,ids,num.conc,flag,spearman,run.ttest,ttest.pval
0,3756,ALL_BUT_MORT,3756 ALL_BUT_MORT,8,4,0.739516,True,0.852840
1,3756,ANY120,3756 ANY120,8,4,0.566306,True,0.896837
2,3756,ANY24,3756 ANY24,8,4,0.727393,True,0.792745
3,3756,AXIS,3756 AXIS,8,4,0.357143,True,0.900140
4,3756,BRN_,3756 BRN_,8,1,NaN,False,NaN
5,3756,CRAN,3756 CRAN,8,4,0.357143,True,0.900140
6,3756,DNC_,3756 DNC_,8,4,0.247436,True,1.000000
7,3756,DP24,3756 DP24,8,2,0.805118,False,NaN
8,3756,EDEM,3756 EDEM,8,4,0.357143,True,0.900140
9,3756,LTRK,3756 LTRK,8,1,0.000000,False,NaN


# Morpho Step 3: Select and Run Models

In [50]:
###########################################
## CALCULATE VALUES FOR LOW QUALITY DATA ##
###########################################

low_quality = dose_response[dose_response["ids"].isin(BMD_Flags[BMD_Flags["flag"].isin([0,1])]["ids"])].groupby("ids")

# Calculate low quality metrics and start new BMDS file 
BMDS_LowQual = low_quality.apply(lambda df: np.trapz(df["frac.affected"], x = df["conc"])).reset_index().rename(columns = {0: "AUC"})
BMDS_LowQual[["Model", "BMD10", "BMDL", "BMD50"]] = np.nan
BMDS_LowQual["Min_Dose"] = round(low_quality[["ids", "conc"]].min("conc").reset_index()["conc"], 4)
BMDS_LowQual["Max_Dose"] = round(low_quality[["ids", "conc"]].max("conc").reset_index()["conc"], 4)
BMDS_LowQual["AUC_Norm"] = BMDS_LowQual["AUC"] / (BMDS_LowQual["Max_Dose"] - BMDS_LowQual["Min_Dose"])

# Order columns
BMDS_LowQual = BMDS_LowQual[["ids", "Model", "BMD10", "BMDL", "BMD50", "AUC", "Min_Dose", "Max_Dose", "AUC_Norm"]]

################
## RUN MODELS ##
################

# Define a function to run models
def select_and_run_models(ID):
    '''For each ID, run the 8 regression models and return the best one.'''
    
    Data = dose_response[dose_response["ids"] == ID]

    # Get the number of nonNA samples per dose
    NonNATotals = Data["num.nonna"].tolist() 

    # Regression model function
    def run_regression_model(modelfun, fittedfun):
        '''Fit the regression model and return the parameters, fitted_values, and the p_value'''

        # Run the model
        model = modelfun(Data[["conc", "num.affected", "num.nonna"]].astype('float').copy())

        # Get the model parameters
        model_params = model.fit().params

        # Get the model's fitted values
        model_fittedvals = fittedfun(Data["conc"], model_params)

        # Get the p_value
        model_pval = calc_p_value(model_fittedvals, model_params)

        # Get the AIC
        AIC = -2*model.fit().llf + (2 * len(model_params))

        # Return a list
        return([model, model_params, model_fittedvals, model_pval, AIC])

    # Calculate P-Value Function
    def calc_p_value(PredictedValues, Params):
        '''Return a p-value of model fit for each unique ID and Model dataframe pairing'''

        # Get the experimental values 
        ExperimentalValues = Data["frac.affected"].tolist()

        # Now, calculate the chi squared value
        ChiSquared = ((NonNATotals / (PredictedValues * (1 - PredictedValues))) * (ExperimentalValues - PredictedValues)**2).sum()

        # Calculate a p-value of fit 
        return(1 - stats.chi2.sf(ChiSquared, len(NonNATotals) - len(Params)))

    # Run regression models in a dictionary
    models = {

        ## Logistic ##
        "Logistic": run_regression_model(baf.Logistic, baf.logistic_fun),

        ## Gamma ## 
        "Gamma": run_regression_model(baf.Gamma, baf.gamma_fun),

        ## Weibull ##
        "Weibull": run_regression_model(baf.Weibull, baf.weibull_fun),

        ## Log-Logistic ##
        "Log Logistic": run_regression_model(baf.Log_Logistic, baf.log_logistic_fun),

        ## Probit ##
        "Probit": run_regression_model(baf.Probit, baf.probit_fun),

        ## Log-Probit ##
        "Log Probit": run_regression_model(baf.Log_Probit, baf.log_probit_fun),

        ## Multistage ##
        "Multistage": run_regression_model(baf.Multistage_2, baf.multistage_2_fun),

        ## Quantal Linear ##
        "Quantal Linear": run_regression_model(baf.Quantal_Linear, baf.quantal_linear_fun),

    }

    # Iterate through all p-values 
    p_values = {}
    for key in models.keys():
        p_values[key] = models[key][3]

    # Determine the best model
    BestModel = min(p_values, key=lambda k: p_values[k]) 

    # Return results 
    return(
        [
            p_values,
            models[BestModel],
            BestModel
        ]
    )

# Run the models 
model_results = {}
for id in BMD_Flags[BMD_Flags["flag"].isin([2,3,4,5])]["ids"]:
    model_results[id] = select_and_run_models(id)

# Make p-value dataframe
p_value_list = []
for id in model_results.keys():
    theDict = model_results[id][0]
    theDict["ids"] = id
    p_value_list.append(theDict)

p_value_df = pd.DataFrame(p_value_list)
p_value_df

,Logistic,Gamma,Weibull,Log Logistic,Probit,Log Probit,Multistage,Quantal Linear,ids
0,0.302375,0.343282,0.342182,0.339346,0.654932,0.322780,0.999844,0.234116,3756 ALL_BUT_MORT
1,0.908758,0.894676,1.000000,0.834579,0.999994,1.000000,1.000000,0.895949,3756 ANY120
2,0.490276,0.562649,0.566872,0.557479,0.773233,0.524819,0.999984,0.453320,3756 ANY24
3,0.775776,0.829327,1.000000,0.830275,0.885261,1.000000,0.999931,0.752342,3756 AXIS
4,0.775776,0.829327,1.000000,0.830275,0.885261,1.000000,0.999931,0.752342,3756 CRAN
5,0.560305,0.781167,0.781167,0.463691,NaN,NaN,0.997187,0.339738,3756 DNC_
6,0.503435,0.588175,1.000000,0.588090,NaN,0.566911,0.993005,0.480834,3756 DP24
7,0.775776,0.829327,1.000000,0.830275,0.885261,1.000000,0.999931,0.752342,3756 EDEM
8,0.223887,0.326651,0.327775,0.326106,0.499286,0.610353,0.999862,0.219538,3756 MO24
9,0.982361,0.939979,1.000000,0.927536,NaN,1.000000,1.000000,0.980956,3756 MORT


In [64]:
def Calculate_BMD(Model, params, BenchmarkResponse = 0.1):
    '''Calculate a benchmark dose'''

    # For each model, extract the parameters and run the calculations
    if (Model == "Logistic"): 
        alpha_, beta_ = params
        return(np.log((1 + np.exp(-alpha_)*BenchmarkResponse)/(1-BenchmarkResponse))/beta_)
    elif (Model == "Gamma"):
        g_, alpha_, beta_ = params
        return(stats.gamma.ppf(BenchmarkResponse, alpha_, scale = 1/beta_))
    elif (Model == "Weibull"):
        g_, alpha_, beta_ = params
        return((-np.log(1 - BenchmarkResponse)/beta_)**(1/alpha_))
    elif (Model == "Log Logistic"):
        g_, alpha_, beta_ = params
        return(np.exp((np.log(BenchmarkResponse/(1-BenchmarkResponse)) - alpha_)/beta_))
    elif (Model == "Probit"):
        alpha_, beta_ = params
        p_0 = stats.norm.cdf(alpha_)
        p_BMD = p_0 + (1 - p_0)*BenchmarkResponse
        return((stats.norm.ppf(p_BMD) - alpha_)/beta_)
    elif (Model == "Log Probit"):
        g_, alpha_, beta_ = params
        return(np.exp((stats.norm.ppf(BenchmarkResponse) - alpha_)/beta_))
    elif (Model == "Multistage"):
        g_, alpha_, beta_ = params
        return((-beta_ + np.sqrt((beta_**2) - (4*beta2_*np.log(1 - BMR))))/(2*beta2_))
    elif (Model == "Quantal Linear"):
        g_, beta_ = params
        return(-np.log(1 - BenchmarkResponse)/beta_)
    else:
        print(Model, "was not recognized as an acceptable model choice.")

def Calculate_BMDL(Model, FittedModelObj, Data, BMD10, params, MaxIterations = 100, ToleranceThreshold = 1e-4):
    '''Run BMDL Function Test'''

    # Reformat data
    Data = Data[["conc", "num.affected", "num.nonna"]].astype('float').copy()

    # Define an initial low and high threshold
    BMD_Low = BMD10/10
    BMD_High = BMD10

    # Start a counter and set tolerance to 1
    Iteration_Count = 0
    Tolerance = 1

    # Set a LLV Threhold
    BMDL_LLV_Thresh = FittedModelObj.fit().llf - stats.chi2.ppf(0.9, 1)/2

    # Start a while condition loop
    while ((Tolerance > ToleranceThreshold) and (Iteration_Count <= MaxIterations)):
        
        # Add to the iteration counters 
        Iteration_Count+=1

        # If maximum iterations are reached, set BMDL to NA and break
        if (Iteration_Count == MaxIterations):
            BMDL = np.nan
            break

        # BMDL should be the mid value between the low and high estimate
        BMDL = (BMD_Low + BMD_High)/2
        ModelObj = np.nan

        # Select the correct BMD model
        if (Model == "Logistic"):
            ModelObj = baf.Logistic_BMD(Data).profile_ll_fit([params[0], BMDL]) # Value is alpha
        elif (Model == "Gamma"):
            ModelObj = baf.Gamma_BMD(Data).profile_ll_fit([params[0], params[1], BMDL]) # Value is g and alpha
        elif (Model == "Weibull"):
            ModelObj = baf.Weibull_BMD(Data).profile_ll_fit([params[0], params[1], BMDL]) # Value is g and alpha
        elif (Model == "Log Logistic"):
            ModelObj = baf.Log_Logistic_BMD(Data).profile_ll_fit([params[0], params[2], BMDL]) # Value is g and beta
        elif (Model == "Probit"):
            ModelObj = baf.Probit_BMD(Data).profile_ll_fit([params[0], BMDL]) # Value is alpha
        elif (Model == "Log Probit"):
            ModelObj = baf.Log_Probit_BMD(Data).profile_ll_fit([params[0], params[1], BMDL]) # Value is g and alpha
        elif (Model == "Multistage"):
            ModelObj = baf.Multistage_2_BMD(Data).profile_ll_fit([params[0], params[1], BMDL]) # Value is g and beta 1
        elif (Model == "Quantal Linear"):
            ModelObj = baf.Quantal_Linear_BMD(Data).profile_ll_fit([params[0], BMDL]) # Value is g
        else:
            print(Model, "was not recognized as an acceptable model choice.")

        # Pull the llf 
        LLF = ModelObj.llf

        # If the calculated LLF is not within the threshold, set high to BMDL and run again 
        if((LLF - BMDL_LLV_Thresh) > 0):
            BMD_High = BMDL
        # Otherwise, set low to BMDL
        else:
            BMD_Low = BMDL

        Tolerance = abs(LLF - BMDL_LLV_Thresh)

    return(BMDL)


# Build BMD table for fitted data 
BMDS_Model = []

for id in model_results.keys():

    # Get the model name 
    Model =  model_results[id][2]

    # Get the fitted modeol object
    FittedModelObj = model_results[id][1][0]

    # Get the parameters 
    params = model_results[id][1][1]

    # Get the BMD10 value 
    BMD10 = Calculate_BMD(Model, params, 0.1)

    # Get the dose response data
    Data = dose_response[dose_response["ids"] == id]

    # Get the AUC, min, and max dose 
    AUC = np.trapz(Data["frac.affected"], x = Data["conc"])
    Min_Dose = round(min(Data["conc"]), 4)
    Max_Dose = round(max(Data["conc"]), 4)

    # Return results in a dictionary
    rowDict = {
        "ids": id,
        "Model": Model,
        "BMD10": BMD10, 
        "BMDL": Calculate_BMDL(Model, FittedModelObj, Data, BMD10, params),
        "BMD50": Calculate_BMD(Model, params, 0.5),
        "AUC": AUC,
        "Min_Dose": Min_Dose,
        "Max_Dose": Max_Dose,
        "AUC_Norm": AUC / (Max_Dose - Min_Dose)
    }
    BMDS_Model.append(rowDict)

BMDS_Model = pd.DataFrame(BMDS_Model)
BMDS_Model

,ids,Model,BMD10,BMDL,BMD50,AUC,Min_Dose,Max_Dose,AUC_Norm
0,3756 ALL_BUT_MORT,Quantal Linear,100.870848,52.520072,663.610495,9.617500,0.0,100.0,0.096175
1,3756 ANY120,Log Logistic,36.500441,NaN,14600.985870,15.625278,0.0,100.0,0.156253
2,3756 ANY24,Quantal Linear,57.430877,34.438108,377.827027,15.699722,0.0,100.0,0.156997
3,3756 AXIS,Quantal Linear,153.476832,62.585087,1009.695452,8.139358,0.0,100.0,0.081394
4,3756 CRAN,Quantal Linear,153.476832,62.585087,1009.695452,8.139358,0.0,100.0,0.081394
5,3756 DNC_,Quantal Linear,1085.560692,297.290178,7141.701312,0.602778,0.0,100.0,0.006028
6,3756 DP24,Quantal Linear,107.930739,61.374738,710.056198,5.830038,0.0,100.0,0.058300
7,3756 EDEM,Quantal Linear,153.476832,62.585087,1009.695452,8.139358,0.0,100.0,0.081394
8,3756 MO24,Quantal Linear,125.579426,56.662121,826.163620,10.764444,0.0,100.0,0.107644
9,3756 MORT,Log Logistic,45.852837,4.742707,445811.838565,10.323276,0.0,100.0,0.103233


In [72]:
# Start final BMDS data frame 
BMDS_Final = pd.concat([BMDS_LowQual, BMDS_Model]).merge(BMD_Flags[["ids", "flag"]]).rename(columns = {"flag":"DataQC_Flag"})

# Add BMD_Analysis flags
BMDS_Final["BMD_Analysis_Flag"] 

,ids,Model,BMD10,BMDL,BMD50,AUC,Min_Dose,Max_Dose,AUC_Norm,Data QC_Flag
0,3756 BRN_,NaN,NaN,NaN,NaN,0.000000,0.0,100.0,0.000000,1
1,3756 LTRK,NaN,NaN,NaN,NaN,2.699267,0.0,100.0,0.026993,1
2,3756 NC__,NaN,NaN,NaN,NaN,0.000000,0.0,100.0,0.000000,1
3,3756 SKIN,NaN,NaN,NaN,NaN,0.000000,0.0,100.0,0.000000,1
4,3756 SM24,NaN,NaN,NaN,NaN,0.000000,0.0,100.0,0.000000,1
5,3756 ALL_BUT_MORT,Quantal Linear,100.870848,52.520072,663.610495,9.617500,0.0,100.0,0.096175,4
6,3756 ANY120,Log Logistic,36.500441,NaN,14600.985870,15.625278,0.0,100.0,0.156253,4
7,3756 ANY24,Quantal Linear,57.430877,34.438108,377.827027,15.699722,0.0,100.0,0.156997,4
8,3756 AXIS,Quantal Linear,153.476832,62.585087,1009.695452,8.139358,0.0,100.0,0.081394,4
9,3756 CRAN,Quantal Linear,153.476832,62.585087,1009.695452,8.139358,0.0,100.0,0.081394,4


In [46]:
# Add missing columns 
#BMDS_LowQual = BMDS_LowQual.merge(min_dose).merge(max_dose).merge(BMD_Flags[["ids", "flag"]]).rename(columns = {"flag":"Data QC_Flag"})
#BMDS_LowQual[["Model", "BMD10", "BMDL", "BMD50", "BMD_Analysis_Flag", "BMD10_Flag", "BMD50_Flag"]] = np.nan

# Reorder columns
#BMDS_LowQual = BMDS_LowQual[["ids", "Model", "BMD10", "BMDL", "BMD50", "AUC", "Min_Dose", "Max_Dose", "AUC_Norm", "Data QC_Flag", 
#                             "BMD_Analysis_Flag", "BMD10_Flag", "BMD50_Flag"]]

0.0

In [73]:
BMD_Flags

,chemical.id,endpoint,ids,num.conc,flag,spearman,run.ttest,ttest.pval
0,3756,ALL_BUT_MORT,3756 ALL_BUT_MORT,8,4,0.739516,True,0.852840
1,3756,ANY120,3756 ANY120,8,4,0.566306,True,0.896837
2,3756,ANY24,3756 ANY24,8,4,0.727393,True,0.792745
3,3756,AXIS,3756 AXIS,8,4,0.357143,True,0.900140
4,3756,BRN_,3756 BRN_,8,1,NaN,False,NaN
5,3756,CRAN,3756 CRAN,8,4,0.357143,True,0.900140
6,3756,DNC_,3756 DNC_,8,4,0.247436,True,1.000000
7,3756,DP24,3756 DP24,8,2,0.805118,False,NaN
8,3756,EDEM,3756 EDEM,8,4,0.357143,True,0.900140
9,3756,LTRK,3756 LTRK,8,1,0.000000,False,NaN
